In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel("Clean.xlsx")

df.head(3)

,Unnamed: 0,ชื่อผู้ใช้ (ผู้ซื้อ),วันที่ทำการสั่งซื้อ,เวลาการชำระสินค้า,ช่องทางการชำระเงิน (รายละเอียด),แผนการผ่อนชำระ,ตัวเลือกการจัดส่ง,*หมายเลขติดตามพัสดุ,วันที่คาดว่าจะทำการจัดส่งสินค้า,เวลาส่งสินค้า,...,Day of Week,DayOfWeek_sin,Time of Day_Evening,Time of Day_Morning,Time of Day_Night,Purchase frequency,Loyalty-Driven Customers,Convenience-Oriented Customers,Price-Sensitive Customers,Profit/Unit
0,0,cariperr.timm,2023-01-02 13:26:00,2023-01-02 13:27:00,NaN,Airpay GIRO,Standard Delivery - ส่งธรรมดาในประเทศ-SPX Express,SPXTH038051184011,2023-01-05 13:27,2023-01-03 17:18,...,0,0.000000,False,False,False,1.0,"High Price , New Customer",Mid Day Customer,False,35.0
1,1,suchanat_niraharnee,2023-01-03 10:06:00,2023-01-03 10:07:00,NaN,Airpay Wallet V2,Standard Delivery - ส่งธรรมดาในประเทศ-Kerry,SHP5317703539,2023-01-05 10:07,2023-01-03 13:01,...,1,0.781831,False,True,False,3.0,"High Price , New Customer",Night Customer,False,54.0
2,2,suchanat_niraharnee,2023-01-03 10:06:00,2023-01-03 10:07:00,NaN,Airpay Wallet V2,Standard Delivery - ส่งธรรมดาในประเทศ-Kerry,SHP5317703539,2023-01-05 10:07,2023-01-03 13:01,...,1,0.781831,False,True,False,3.0,"High Price , New Customer",Night Customer,False,54.0


In [3]:
df.columns

Index(['Unnamed: 0', 'ชื่อผู้ใช้ (ผู้ซื้อ)', 'วันที่ทำการสั่งซื้อ',
       'เวลาการชำระสินค้า', 'ช่องทางการชำระเงิน (รายละเอียด)',
       'แผนการผ่อนชำระ', 'ตัวเลือกการจัดส่ง', '*หมายเลขติดตามพัสดุ',
       'วันที่คาดว่าจะทำการจัดส่งสินค้า', 'เวลาส่งสินค้า', 'ชื่อสินค้า',
       'เลขอ้างอิง SKU (SKU Reference No.)', 'ชื่อตัวเลือก', 'ราคาตั้งต้น',
       'Sale Price', 'จำนวน', 'Returned quantity', 'ราคาขายสุทธิ',
       'Shopee Discount', 'Discount Code by Seller', 'Code Coins Cashback',
       'โค้ดส่วนลดชำระโดย Shopee', 'โค้ดส่วนลด',
       'เข้าร่วมแคมเปญ bundle deal หรือไม่',
       'ส่วนลด bundle deal ชำระโดยผู้ขาย', 'Shopee-Paid Bundle Deal Discount',
       'ส่วนลดจากการใช้เหรียญ', 'ส่วนลดทั้งหมดจากบัตรเครดิต', 'ค่าคอมมิชชั่น',
       'Transaction Fee', 'ต้นทุนขายหักคูปองและcoin',
       'ค่าจัดส่งที่ชำระโดยผู้ซื้อ', 'ค่าจัดส่งที่ Shopee ออกให้โดยประมาณ',
       'ค่าจัดส่งสินค้าคืน', 'ค่าบริการ', 'จำนวนเงินทั้งหมด',
       'ค่าจัดส่งโดยประมาณ', 'หมายเหตุจากผู้ซื้อ', 'ประเทศ', 'จั

In [4]:
df = df[["ชื่อสินค้า" , "ราคาตั้งต้น" , "ราคาขายสุทธิ" , "Discount Code by Seller" , "ต้นทุนขายหักคูปองและcoin" , "ค่าจัดส่งโดยประมาณ" , "ค่าจัดส่งที่ Shopee ออกให้โดยประมาณ"]]
df.head(3)

,ชื่อสินค้า,ราคาตั้งต้น,ราคาขายสุทธิ,Discount Code by Seller,ต้นทุนขายหักคูปองและcoin,ค่าจัดส่งโดยประมาณ,ค่าจัดส่งที่ Shopee ออกให้โดยประมาณ
0,ชุดถาดลูกกลิ้ง ชุดถาดทาสี + เครื่องมือ 5 ชิ้น ...,250,199,5,194,35,0
1,จระเข้ กาวยาแนว ยาแนว จระเข้เงิน ( น้ำหนัก 1 K...,30,22,5,131,54,0
2,เกรียง เกรียงพลาสติกอเนกประสงค์ เกียงยาแนว AT ...,80,59,5,131,54,0


In [5]:
def compute_profit_margin(x):
    net_re = x["ราคาขายสุทธิ"] - x["Discount Code by Seller"]
    tax = 0.07 * (x["ราคาขายสุทธิ"] - x["ต้นทุนขายหักคูปองและcoin"])
    me = (x["ราคาขายสุทธิ"] * (0.0749 + 0.0535 + 0.0535)) + (0.0321 * net_re) # + x["ค่าจัดส่งโดยประมาณ"] - x["ค่าจัดส่งที่ Shopee ออกให้โดยประมาณ"]
    gp = net_re - x["ต้นทุนขายหักคูปองและcoin"] - tax
    ni = gp - me
    x["Profit Margin"] = (ni/x["ต้นทุนขายหักคูปองและcoin"]) * 100

    return x

df = df.apply(compute_profit_margin , axis = 1)

df.head(3)

ZeroDivisionError: float division by zero

In [ ]:
def compute_profit_margin(x):
    net_re = x["ราคาขายสุทธิ"] - x["Discount Code by Seller"]
    tax = 0.07 * (x["ราคาขายสุทธิ"] - x["ต้นทุนขายหักคูปองและcoin"])
    me = (x["ราคาขายสุทธิ"] * (0.0749 + 0.0535 + 0.0535)) + (0.0321 * net_re) # + x["ค่าจัดส่งโดยประมาณ"] - x["ค่าจัดส่งที่ Shopee ออกให้โดยประมาณ"]
    gp = net_re - x["ต้นทุนขายหักคูปองและcoin"] - tax
    ni = gp - me
    x["Profit Margin"] = (ni/x["ต้นทุนขายหักคูปองและcoin"]) * 100

    return x

In [22]:
df[df["ต้นทุนขายหักคูปองและcoin"] == 0]

,ราคาตั้งต้น,ราคาขายสุทธิ,Discount Code by Seller,ต้นทุนขายหักคูปองและcoin,ค่าจัดส่งโดยประมาณ,ค่าจัดส่งที่ Shopee ออกให้โดยประมาณ,Profit Margin
3341,25.0,19.0,0.0,0.0,27.0,27.0,inf
3373,12.0,20.0,0.0,0.0,27.0,27.0,inf
3384,19.0,15.0,0.0,0.0,27.0,27.0,inf
3534,9.0,14.0,0.0,0.0,37.0,30.0,inf
4389,75.0,117.0,5.0,0.0,103.0,0.0,inf
4941,35.0,30.0,0.0,0.0,42.0,0.0,inf


In [17]:
# df.to_excel("Clean.xlsx" , index = False)